# Wave with periodic bc with the scheme from Palma

We solve the wave equation with Dirichlet boundary conditions (homogeneous) up to $T = 2L$ where $L$ is the space size of the integrating region. The data is reflection symmetric, $u_0(x) = u_0(L-x)$, $\partial_t u(x,t=0) = 0$. This way at the last point the solution repeats itself and we can measure the error very well.

In [1]:
import Pkg
#Pkg.add("Zygote")
#Pkg.instantiate()
#Pkg.add("StatsBase")
#Pkg.add("UnPack")
#Pkg.add("Revise")
using ComponentArrays
using Distributions
#using WGLMakie
using Lux
using LuxCUDA
using OptimizationOptimJL
using Random
using UnPack
using Zygote
#using CairoMakie  # Backend que funciona en notebooks
using StatsBase
using Revise
using Plots
using JLD2
#import NaNMath

## Status:

Working well in both GPU and CPU without adaptivity. Using both initial data for u and its derivative.

I put some functions here.

In [2]:
includet("../../neural_tools.jl")

In [3]:
# -------------------------------------------------------------------
# Configuración
# -------------------------------------------------------------------
config = Dict(
    :N_input => 2,          # [x; t]
    :N_neurons => 20,
    :N_layers => 3,
    :N_output => 1, 
    :N_points => 15_000,     # puntos de colisión (x,t)
    :N_points_bound => 0, # puntos de frontera
    :N_points_0 => 0,    # puntos de condición inicial
    :xmin => 0.0,
    :xmax => 1.0,           # = L dominio espacial
    :tmin => 0.0,           # t_min
    :tmax => 2.0,           # t_max
    #:optimizer => BFGS(),
    :optimizer => SSBroyden(),
    :maxiters => 3_000,
    :N_rounds => 20,    # 5           # nº de rondas RAD
    :iters_per_round => 500,       # iteraciones BFGS por ronda
    :k1 => 1.0, 
    :k2 => 1.0,        # hiperparámetros RAD
    :N_test => 30_000,             # candidatos por ronda, mayor que N_points
    #:method => :adaptive,
    :method => :direct,
    # for the initial data
    :A => 1.0,
    :B => 1.0,
    :x0 => 2.0,
    :x1 => 3.0,
    :p => 2,
    :c => 1.0
)

Dict{Symbol, Any} with 25 entries:
  :N_input         => 2
  :N_test          => 30000
  :N_points_bound  => 0
  :iters_per_round => 500
  :N_points_0      => 0
  :N_points        => 15000
  :N_neurons       => 20
  :k2              => 1.0
  :x0              => 2.0
  :A               => 1.0
  :N_output        => 1
  :tmax            => 2.0
  :maxiters        => 3000
  :p               => 2
  :B               => 1.0
  :c               => 1.0
  :N_layers        => 3
  :method          => :direct
  :xmin            => 0.0
  ⋮                => ⋮

**Note 1:** I attempted to put a bump function of compact support (with some if's) and could not make it run. It seems there are problems with the AD scheme to handle if's. So I put a simpler function.

**Note 2:** As it is, with adaptive it takes 12 hours (CPU) to run... And the solution is wrong...

In [4]:
# -------------------------------------------------------------------
# Second order wave equation (onda 1D): u_tt - c^2 u_xx = 0
# -------------------------------------------------------------------
wave_equation(∂2u_∂x2, ∂2u_∂t2, c) = ∂2u_∂t2 .- (c^2) .* ∂2u_∂x2

```
Residual at collocation points
``` 

function residual_at_points_Dirichlet(input, NN, Θ, st)
    x, t = input[1:1, :], input[2:2, :]
    _, u_xx, u_tt = calculate_derivatives_Dirichlet(x, t, NN, Θ, st)
    res = wave_equation(u_xx, u_tt, config[:c])
    return vec(abs.(res |> cpu_device()))  # magnitud del residuo en CPU
end


# -------------------------------------------------------------------
# Loss function with Dirichlet boundary conditions hard enforced.
# -------------------------------------------------------------------

function loss_function(input, NN, Θ, st)
    res = residual_at_points_Dirichlet(input[1], NN, Θ, st)
    #return NaNMath.log10(sum(abs2, res) / length(res))
    return log10(sum(abs2, res) / length(res))
end

#=
function loss_function(input, NN, Θ, st)
    x, t = input[1:1, :], input[2:2, :]
    _, u_xx, u_tt = calculate_derivatives_Dirichlet(x, t, NN, Θ, st)
    res = wave_equation(u_xx, u_tt, config[:c])
    #return NaNMath.log10(sum(abs2, res) / length(res))
    return log10(sum(abs2, res) / length(res))
end
=#

# -------------------------------------------------------------------
# Callback
# -------------------------------------------------------------------
function callback(p, l, losses)
    push!(losses, l)
    println("Current loss: ", l)
    return false
end

callback (generic function with 1 method)

Initial data:

In [5]:
#This way it does not work on GPUS, so we define the function inside the other function
#u0(x) = config[:A]*(x .- xmin).^4 .* (x .- xmax).^4 ./ ((xmax - xmin)/2)^8 # Initial condition
#u1(x) = config[:B]*(x .- xmin).^4 .* (x .- xmax).^4 ./ ((xmax - xmin)/2)^8 # Initial condition for the time derivative

@unpack xmin, xmax = config
u0(x) = (x .- xmin).^4 .* (x .- xmax).^4 ./ ((xmax - xmin)/2)^8
u1(x) = (x .- xmin).^3 .* (x .- xmax).^3 ./ ((xmax - xmin)/2)^8 .* (2x .- (xmax - xmin))

u1 (generic function with 1 method)

In [6]:
# -------------------------------------------------------------------
# Entrenamiento
# -------------------------------------------------------------------
losses = Float64[]
NN, Θ, st = create_neural_network(config)
input = generate_input_x_t(config)
input0 = generate_input0_x(config)
input_bound = generate_input_boundary_x(config)
input_total = [input, input0, input_bound]

@show typeof(input_total) size(input_total)

input_total[1] === input

Total params: 1341


┌ Warning: No functional GPU backend found! Defaulting to CPU.
│ 
│ 1. If no GPU is available, nothing needs to be done. Set `MLDATADEVICES_SILENCE_WARN_NO_GPU=1` to silence this warning.
│ 2. If GPU is available, load the corresponding trigger package.
│     a. `CUDA.jl` and `cuDNN.jl` (or just `LuxCUDA.jl`) for  NVIDIA CUDA Support.
│     b. `AMDGPU.jl` for AMD GPU ROCM Support.
│     c. `Metal.jl` for Apple Metal GPU Support. (Experimental)
│     d. `oneAPI.jl` for Intel oneAPI GPU Support. (Experimental)
└ @ MLDataDevices.Internal /Users/reula/.julia/packages/MLDataDevices/NeohJ/src/internal.jl:112


typeof(input_total) = Vector{Array{Float64}}
size(input_total) = (3,)


true

In [7]:
input_total[2]

3-element Vector{Float64}:
 0.0
 0.0
 0.0

In [8]:
loss_function(input_total, NN, Θ, st)

3.4094659192855152

In [ ]:


#calculate_Dirichlet_f(input[1:1, :], input[2:2, :], NN, Θ, st)
#f, ∂2f_∂x2, ∂2f_∂t2 = calculate_derivatives_Dirichlet(input[1:1, :], input[2:2, :], NN, Θ, st)

In [51]:

adaptive = true
#adaptive = false

if adaptive 
    # Configura las rondas adaptativas (ajusta a tu gusto)
    nrounds = 20                # nº de rondas RAD
    iters_per_round = 500       # iteraciones BFGS por ronda
    #k1, k2 = 1.0, 1          # hiperparámetros RAD
    Ntest = 200_000              # candidatos por ronda

    for r in 1:nrounds
        @info "RAD round $r / $nrounds  |  iters=$iters_per_round"
        # Optimiza sobre el conjunto actual de colisión
        optf   = OptimizationFunction((Θ, input) -> loss_function(input, NN, Θ, st), AutoZygote())
        optprob = OptimizationProblem(optf, Θ, input)
        optres  = solve(
            optprob,
            config[:optimizer];
            callback = (p, l) -> callback(p, l, losses),
            maxiters = iters_per_round,
        )
        Θ = optres.u  # continúa desde el óptimo de la ronda

        # Re-muestrea puntos de colisión ponderando por residuo
        input = adaptive_rad(NN, Θ, st, config; Ntest=Ntest, Nint=config[:N_points])#, k1=k1, k2=k2)
    end

else
    @info "Training with $(config[:N_points]) collocation points and $(config[:maxiters]) iterations"

    optf = OptimizationFunction((Θ, input) -> loss_function(input, NN, Θ, st), AutoZygote())
    optprob = OptimizationProblem(optf, Θ, input)

    optresult = solve(
        optprob,
        callback = (p, l) -> callback(p, l, losses),
        config[:optimizer],
        maxiters = config[:maxiters],
    )

    Θ = optresult.u

    input = generate_input_t_x(config)  # reset input

end

# Parámetros optimizados a CPU si procede
Θ = optresult.u |> cpu_device()




Current loss: 3.4015428524120073

┌ Info: RAD round 1 / 20  |  iters=500
└ @ Main /Users/reula/Julia/PINN/PINNS_codes/SimplePoissonPINN/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X20sZmlsZQ==.jl:13



Current loss: 3.3981067247896073
Current loss: 3.396870473360334
Current loss: 3.3945634181232345
Current loss: 3.384222566157975
Current loss: 3.361779691575747
Current loss: 3.3469304995103895
Current loss: 3.3278368505792066
Current loss: 3.317218636224496
Current loss: 3.259332824571138
Current loss: 3.2483394073335203
Current loss: 3.2269018977216364
Current loss: 3.172723629836581
Current loss: 3.0794784152751564
Current loss: 2.9823266609307053
Current loss: 

InterruptException: Error trying to display an error.

In [ ]:
Θ, st, losses = compute_solution_1d(config, input_total, NN, Θ, st)
# Parámetros optimizados a CPU si procede
Θ = optresult.u |> cpu_device()

┌ Info: Normal training round 1 / 20  |  iters=500
└ @ Main /Users/reula/Julia/PINN/neural_tools.jl:467


Current loss: 3.4094659192855152
Current loss: 3.404601130913995
Current loss: 3.3998334923671205
Current loss: 3.395770435081597
Current loss: 3.3567363081691863
Current loss: 3.3530114625967586
Current loss: 3.33952020697565
Current loss: 3.324570511118808
Current loss: 3.320174084401749
Current loss: 

In [ ]:
println("Training completed. Saving data")
@save "wave_Dir_1.jld2" config Θ st losses

In [ ]:
plot(losses, label = "Loss", title = "loss vs iterations")


In [ ]:

t = 1.0/4
xs = reshape(collect(range(config[:xmin], config[:xmax], length=200)), 1, :)
t_fix = reshape(fill(t, length(xs)), 1, :)
#sol_p = [calculate_Dirichlet_f(xs, t_fix, NN, Θ, st) for x in xs]
sol_p = [calculate_Dirichlet_f(xs[:,i], t_fix[:,i], NN, Θ, st)[1] for i in 1:length(xs[1,:])]

plot(xs[1,:],sol_p, label = "t=$t", title = "solution")
#lines!(xs[1,:],bump.(xs[1,:], config[:x0], config[:x1], config[:p], config[:A]), label = "initial condition")
plot!(xs[1,:],256*config[:A]*(xs[1,:] .- config[:xmin]).^4 .* (xs[1,:] .- config[:xmax]).^4, label = "initial condition", linestyle = :dash)     


In [ ]:
initad = [-config[:A]*(xs[1,i] .- config[:xmin]).^4 .* (xs[1,i] .- config[:xmax]).^4 for i in 1:length(xs[1,:])]
S = maximum(-sol_p)/maximum(-initad)

In [ ]:
ax = (title = "solution")
t = 0.0
xs = reshape(collect(range(config[:xmin], config[:xmax], length=200)), 1, :)
t_fix = reshape(fill(t, length(xs)), 1, :)
#sol_p = [calculate_Dirichlet_f(xs, t_fix, NN, Θ, st) for x in xs]
sol_p = [calculate_Dirichlet_f(xs[:,i], t_fix[:,i], NN, Θ, st)[1] for i in 1:length(xs[1,:])]

plot(xs[1,:],sol_p .+ (config[:A]*(xs[1,:] .+ config[:xmin]).^4 .* (xs[1,:] .- config[:xmax]).^4), label = "t=0")
#plot!(xs[1,:],bump.(xs[1,:], config[:x0], config[:x1], config[:p], config[:A]), label = "initial condition")
plot!(xs[1,:],sol_p .-config[:A]*(xs[1,:] .- config[:xmin]).^4 .* (xs[1,:] .- config[:xmax]).^4, label = "initial condition", linestyle = :dash)     

**Save data:**

In [ ]:
#@save "wave_dirichlet_1D_adaptive.jld2" NN Θ st config losses
@save "wave_dirichlet_1D.jld2" NN Θ st config losses